In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy 

import scipy.interpolate as interpolate
import scipy.constants as constants

In [4]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [5]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

### Setup parameters

In [6]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [7]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [8]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [9]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.24060921459508
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.683293718069
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308046279
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.81841601739336
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243975466
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.238026011364
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478568309
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591713835
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737392779
*******************

In [10]:
print("check if scale is positive: ", gfw.scale_Z2)
gfw.scale_Z2 *= -1

check if scale is positive:  -1.84743267861e+17


In [11]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [12]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = 10
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 870.25it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 10


In [595]:
def func_filter(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    res = bb.z2
    return (res)

In [596]:
test_phases = np.array(Parallel(n_jobs=46)(delayed(func_filter)(i) for i in tqdm(range(10000))))









  0%|          | 0/10000 [00:00<?, ?it/s]







  0%|          | 1/10000 [00:02<7:59:44,  2.88s/it]







  1%|          | 92/10000 [00:04<5:33:39,  2.02s/it]







  1%|▏         | 138/10000 [00:05<3:52:59,  1.42s/it]







  2%|▏         | 184/10000 [00:05<2:42:53,  1.00it/s]







  2%|▏         | 230/10000 [00:05<1:53:57,  1.43it/s]







  3%|▎         | 276/10000 [00:06<1:19:52,  2.03it/s]







  3%|▎         | 322/10000 [00:06<56:09,  2.87it/s]  







  4%|▎         | 368/10000 [00:07<39:36,  4.05it/s]







  4%|▍         | 414/10000 [00:07<28:05,  5.69it/s]







  5%|▍         | 460/10000 [00:08<20:03,  7.93it/s]







  5%|▌         | 506/10000 [00:08<14:29, 10.92it/s]







  6%|▌         | 552/10000 [00:09<10:33, 14.91it/s]







  6%|▌         | 598/10000 [00:09<07:50, 19.98it/s]







  6%|▋         | 644/10000 [00:10<05:59, 26.04it/s]







  7%|▋         | 690/10000 [00:10<04:39, 33.28it/s]







  7%|▋         | 736/10000 [00:11<03:43, 41.50i

 45%|████▍     | 4462/10000 [00:49<00:56, 97.60it/s] 







 45%|████▌     | 4508/10000 [00:50<00:54, 100.55it/s]







 46%|████▌     | 4554/10000 [00:50<00:55, 98.11it/s] 







 46%|████▌     | 4600/10000 [00:51<00:54, 98.63it/s]







 46%|████▋     | 4646/10000 [00:51<00:55, 97.30it/s]







 47%|████▋     | 4692/10000 [00:51<00:52, 101.05it/s]







 47%|████▋     | 4738/10000 [00:52<00:54, 96.76it/s] 







 48%|████▊     | 4784/10000 [00:52<00:52, 98.56it/s]







 48%|████▊     | 4830/10000 [00:53<00:52, 98.43it/s]







 49%|████▉     | 4876/10000 [00:53<00:51, 100.26it/s]







 49%|████▉     | 4922/10000 [00:54<00:50, 101.50it/s]







 50%|████▉     | 4968/10000 [00:54<00:49, 101.53it/s]







 50%|█████     | 5014/10000 [00:55<00:50, 99.55it/s] 







 51%|█████     | 5060/10000 [00:55<00:51, 96.70it/s]







 51%|█████     | 5106/10000 [00:56<00:52, 94.03it/s]







 52%|█████▏    | 5152/10000 [00:56<00:49, 98.30it/s]







 52%|█████▏    | 5198/10000 [00

 89%|████████▉ | 8924/10000 [01:35<00:10, 98.22it/s]







 90%|████████▉ | 8970/10000 [01:35<00:10, 96.29it/s]







 90%|█████████ | 9016/10000 [01:36<00:10, 95.53it/s]







 91%|█████████ | 9062/10000 [01:36<00:09, 95.55it/s]







 91%|█████████ | 9108/10000 [01:37<00:09, 96.46it/s]







 92%|█████████▏| 9154/10000 [01:37<00:08, 96.48it/s]







 92%|█████████▏| 9200/10000 [01:38<00:08, 94.61it/s]







 92%|█████████▏| 9246/10000 [01:38<00:07, 97.60it/s]







 93%|█████████▎| 9292/10000 [01:39<00:07, 95.70it/s]







 93%|█████████▎| 9338/10000 [01:39<00:06, 98.98it/s]







 94%|█████████▍| 9384/10000 [01:40<00:06, 96.68it/s]







 94%|█████████▍| 9430/10000 [01:40<00:05, 98.63it/s]







 95%|█████████▍| 9476/10000 [01:40<00:05, 99.12it/s]







 95%|█████████▌| 9522/10000 [01:41<00:04, 101.03it/s]







 96%|█████████▌| 9568/10000 [01:41<00:04, 99.52it/s] 







 96%|█████████▌| 9614/10000 [01:42<00:03, 98.09it/s]







 97%|█████████▋| 9660/10000 [01:42<00:

In [15]:
def func_sideband(x, gravity_freqs_all):
    res = []
    for freq_ in gravity_freqs_all:
        b, a = signal.butter(3, [2.*(freq_+1.5-bandwidth/2.)/5000, 2.*(freq_+1.5+bandwidth/2.)/5000], btype = 'bandpass')
        responsefilt = signal.filtfilt(b, a, x)
        res.append(np.std(responsefilt))
    return np.array(res)

In [37]:
ll = likelihood_analyser.LikelihoodAnalyser()

def get_alpha(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
#     noises_std = np.array([1])
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
        
    return mm_tmp

In [380]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,13)[1:]
# gravity_freqs_all = np.array([36])
G_lambda = 10
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])


fft_angles = np.angle(np.fft.rfft(shaking378_template))
fft_fft = np.abs(np.fft.rfft(shaking378_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    
    gravity_phases.append(phase_all)

gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data


In [18]:
mm_gravity = get_alpha(0)

MLEs:  <ValueView of Minuit at cf2cd48>
  A: -0.6055552453472046
  phi: 0.0
  sigma: 131.1011895204217


### EDM settings

In [19]:
base_path = r'/home/nadavp/tmp/'
fname = 'new_trap_attractor_shield_potential_efield.txt'


positions_to_sample = np.array([
                                [13.9e-6, 0.0e-6, -15.23e-6], \
                               ])

dipole_units = constants.elementary_charge * 1.0e-6
dipole_moment = np.array([0.0, 0.0, 100.0]) * dipole_units


In [20]:
#############################################################
######  LOAD DATA FROM COMSOL OUTPUT TO NUMPY ARRAYS  #######
#############################################################

path = os.path.join(base_path, fname)

with open(path, 'r') as f:
    lines  =  f.readlines()

for lineind, line in enumerate(lines):
    if 'Grid' in line:
        grid_ind = lineind + 1
    elif 'V (V)' in line:
        voltage_ind = lineind + 1
    elif 'es.Ex (V/m)' in line:
        ex_ind = lineind + 1
    elif 'es.Ey (V/m)' in line:
        ey_ind = lineind + 1
    elif 'es.Ez (V/m)' in line:
        ez_ind = lineind + 1

xvec_str = lines[grid_ind].strip('\n')
yvec_str = lines[grid_ind+1].strip('\n')
zvec_str = lines[grid_ind+2].strip('\n')

xvec = np.array( [float(val) for val in xvec_str.split()] )
yvec = np.array( [float(val) for val in yvec_str.split()] )
zvec = np.array( [float(val) for val in zvec_str.split()] )

lenx = len(xvec)
leny = len(yvec)
lenz = len(zvec)

voltage = np.zeros((lenx, leny, lenz))
efieldx = np.zeros((lenx, leny, lenz))
efieldy = np.zeros((lenx, leny, lenz))
efieldz = np.zeros((lenx, leny, lenz))

for j in range(leny):
    for k in range(lenz):
        voltage_line = lines[voltage_ind + j + k*leny]
        voltage[:,j,k] = np.array( [float(val) for val in voltage_line.strip('\n').split()] ) 

        efieldx_line = lines[ex_ind + j + k*leny]
        efieldx[:,j,k] = np.array( [float(val) for val in efieldx_line.strip('\n').split()] )

        efieldy_line = lines[ey_ind + j + k*leny]
        efieldy[:,j,k] = np.array( [float(val) for val in efieldy_line.strip('\n').split()] )

        efieldz_line = lines[ez_ind + j + k*leny]
        efieldz[:,j,k] = np.array( [float(val) for val in efieldz_line.strip('\n').split()] )





#############################################################
##############  BUILD 3D GRID INTERPOLATORS  ################
#############################################################

voltage_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), voltage)

efieldx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldx)
efieldy_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldy)
efieldz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldz)

In [48]:
dipole_moment = np.array([0.0, 0.0, 100.0]) * dipole_units
x_att = [100*np.sin(3*t/500*2*np.pi) for t in range(500)]
positions_to_sample4 = np.array([[13.9*1e-6, (offset_-4.9)*1e-6, -15.23e-6] for offset_ in x_att])

In [49]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x3 = []
force_z3 = []
for point in positions_to_sample4:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x3.append(force_on_dipole[0])
    force_z3.append(force_on_dipole[2])


    MS dipole [e um] :  [   0.    0.  100.]

    MS Postion [um]  :  [ 13.9   -4.9  -15.23]
        Efield [V/m] :  [-3629.04376377   -32.38996578   745.93786973]
Force on dipole [N]  :  [ -2.84274175e-15  -6.11284832e-17  -1.00784708e-15]

    MS Postion [um]  :  [ 13.9         -1.13098173 -15.23      ]
        Efield [V/m] :  [-3613.33808298   -18.74908864   743.87823202]
Force on dipole [N]  :  [ -2.74842919e-15  -1.93147132e-16  -1.18463714e-15]

    MS Postion [um]  :  [ 13.9          2.63268055 -15.23      ]
        Efield [V/m] :  [-3775.95027138    79.85385886   689.03226592]
Force on dipole [N]  :  [ -2.55172345e-15   1.40078774e-15  -2.12120465e-15]

    MS Postion [um]  :  [ 13.9          6.38563849 -15.23      ]
        Efield [V/m] :  [-3560.1701477     28.00474276   811.18059762]
Force on dipole [N]  :  [ -3.16755659e-15  -6.68095120e-16  -1.57725577e-15]

    MS Postion [um]  :  [ 13.9         10.12255891 -15.23      ]
        Efield [V/m] :  [-3586.5531662     -5.99602

    MS Postion [um]  :  [ 13.9        94.5951017 -15.23     ]
        Efield [V/m] :  [-3579.1008388      9.03673029   801.38592255]
Force on dipole [N]  :  [ -2.62015086e-15  -6.85134994e-17  -1.60456673e-15]

    MS Postion [um]  :  [ 13.9         94.90267284 -15.23      ]
        Efield [V/m] :  [ -3.58475748e+03   5.63826311e-01   7.89578818e+02]
Force on dipole [N]  :  [ -2.66748752e-15  -9.24750579e-17  -1.68409581e-15]

    MS Postion [um]  :  [ 13.9         95.06841893 -15.23      ]
        Efield [V/m] :  [ -3.58610225e+03  -2.54027245e+00   7.85677760e+02]
Force on dipole [N]  :  [ -2.67860359e-15  -1.04403206e-16  -1.71049456e-15]

    MS Postion [um]  :  [ 13.9         95.09210442 -15.23      ]
        Efield [V/m] :  [ -3.58594813e+03  -2.68669107e+00   7.85620694e+02]
Force on dipole [N]  :  [ -2.67726627e-15  -1.05907651e-16  -1.71092131e-15]

    MS Postion [um]  :  [ 13.9         94.97369566 -15.23      ]
        Efield [V/m] :  [ -3.58606369e+03  -1.39269505e+00   7.8

    MS Postion [um]  :  [ 13.9         11.36371652 -15.23      ]
        Efield [V/m] :  [ -3.60070624e+03   2.46998204e+00   8.33064308e+02]
Force on dipole [N]  :  [ -3.47312144e-15  -2.00339562e-16  -5.88797986e-16]

    MS Postion [um]  :  [ 13.9          7.63332336 -15.23      ]
        Efield [V/m] :  [-3595.17229866   -46.50943258   758.55351445]
Force on dipole [N]  :  [ -3.24357540e-15  -2.06743600e-16  -9.04390909e-16]

    MS Postion [um]  :  [ 13.9          3.88511966 -15.23      ]
        Efield [V/m] :  [-3789.63764482  -195.09211884   930.66377068]
Force on dipole [N]  :  [ -4.74398423e-15   7.53244287e-16  -1.76637499e-15]

    MS Postion [um]  :  [ 13.9          0.12443182 -15.23      ]
        Efield [V/m] :  [-3631.35270529     5.28589288   746.28098593]
Force on dipole [N]  :  [ -2.66565004e-15  -9.00472223e-17  -9.13314740e-16]

    MS Postion [um]  :  [ 13.9         -3.64339601 -15.23      ]
        Efield [V/m] :  [-3616.73768798   -30.83029278   741.90376849]
Fo

    MS Postion [um]  :  [  13.9        -102.35268728  -15.23      ]
        Efield [V/m] :  [-3417.39986295   -61.42062823   635.70489757]
Force on dipole [N]  :  [ -3.02312377e-15  -5.16610934e-16   8.70259825e-17]

    MS Postion [um]  :  [  13.9        -103.12872507  -15.23      ]
        Efield [V/m] :  [-3387.96236136   -63.61128224   668.23887155]
Force on dipole [N]  :  [ -2.92606017e-15  -2.55636207e-18   1.99392715e-16]

    MS Postion [um]  :  [  13.9        -103.76517447  -15.23      ]
        Efield [V/m] :  [-3369.0966476    -76.98210176   659.16467693]
Force on dipole [N]  :  [ -2.97417995e-15   3.43351344e-16   2.59008481e-17]

    MS Postion [um]  :  [  13.9        -104.26113105  -15.23      ]
        Efield [V/m] :  [-3365.80549671   -80.70798187   656.19162997]
Force on dipole [N]  :  [ -2.99495038e-15   3.91738298e-16  -2.24155537e-16]

    MS Postion [um]  :  [  13.9        -104.61589003  -15.23      ]
        Efield [V/m] :  [-3370.79079202   -79.06761475   656.701

    MS Postion [um]  :  [ 13.9        -33.40192625 -15.23      ]
        Efield [V/m] :  [-3560.34038986   -65.60288661   746.23608462]
Force on dipole [N]  :  [ -3.23098890e-15  -1.33846886e-16  -1.75308720e-15]

    MS Postion [um]  :  [ 13.9        -29.76898872 -15.23      ]
        Efield [V/m] :  [-3493.58389895   -15.41187987   795.01175276]
Force on dipole [N]  :  [ -4.46651248e-15   4.15947723e-16  -2.45893297e-15]

    MS Postion [um]  :  [ 13.9        -26.10071099 -15.23      ]
        Efield [V/m] :  [-3481.72094555   -34.27123803   837.93960662]
Force on dipole [N]  :  [ -4.26611518e-15   2.18581979e-17  -2.22631355e-15]

    MS Postion [um]  :  [ 13.9       -22.4023059 -15.23     ]
        Efield [V/m] :  [-3603.42652995   -40.60965082   766.23705502]
Force on dipole [N]  :  [ -3.62093227e-15  -2.36470342e-16  -8.10605787e-16]

    MS Postion [um]  :  [ 13.9        -18.67902907 -15.23      ]
        Efield [V/m] :  [-3591.09439558   -16.41534073   750.31233761]
Force on di

...

    MS Postion [um]  :  [ 13.9        -56.34395338 -15.23      ]
        Efield [V/m] :  [-3640.49419989   -49.80195663   677.85759035]
Force on dipole [N]  :  [ -3.30684346e-15  -2.43364723e-17  -1.55627345e-15]

    MS Postion [um]  :  [ 13.9        -53.07536741 -15.23      ]
        Efield [V/m] :  [-3588.62998889   -51.05343456   712.43788393]
Force on dipole [N]  :  [ -3.18232281e-15   6.40640890e-17  -1.48500260e-15]

    MS Postion [um]  :  [ 13.9        -49.73832161 -15.23      ]
        Efield [V/m] :  [-3594.13527096   -64.56767859   715.59361748]
Force on dipole [N]  :  [ -2.63198018e-15  -5.27458122e-17  -6.26301605e-16]

    MS Postion [um]  :  [ 13.9       -46.3375581 -15.23     ]
        Efield [V/m] :  [-3553.97853439   -41.13162169   706.99913479]
Force on dipole [N]  :  [ -3.58742494e-15   1.33637129e-16  -1.05108159e-15]

    MS Postion [um]  :  [ 13.9        -42.87790955 -15.23      ]
        Efield [V/m] :  [-3529.40508599     4.94511528   670.86226679]
Force on di

    MS Postion [um]  :  [ 13.9         77.45325976 -15.23      ]
        Efield [V/m] :  [-3543.54560902    14.20298581   772.40606065]
Force on dipole [N]  :  [ -2.64664986e-15   1.05062990e-16  -1.93058141e-15]

    MS Postion [um]  :  [ 13.9         79.53279255 -15.23      ]
        Efield [V/m] :  [-3556.00725354     6.16257933   791.65302661]
Force on dipole [N]  :  [ -2.78352326e-15   1.32659913e-16  -1.79667678e-15]

    MS Postion [um]  :  [ 13.9         81.49234172 -15.23      ]
        Efield [V/m] :  [-3560.06819512    22.53082555   821.87032546]
Force on dipole [N]  :  [ -2.88916647e-15   2.50792979e-16  -1.69071225e-15]

    MS Postion [um]  :  [ 13.9         83.32912264 -15.23      ]
        Efield [V/m] :  [-3555.38576688    16.24829797   840.53673344]
Force on dipole [N]  :  [ -2.95379957e-15  -3.07342308e-18  -1.81636284e-15]

    MS Postion [um]  :  [ 13.9         85.04052516 -15.23      ]
        Efield [V/m] :  [-3572.27884286    18.76000451   833.46509963]
Force on

    MS Postion [um]  :  [ 13.9         56.88596131 -15.23      ]
        Efield [V/m] :  [ -3.64199249e+03  -3.23561943e-01   8.37503280e+02]
Force on dipole [N]  :  [ -3.09828668e-15  -2.71735114e-16  -1.87871198e-16]

    MS Postion [um]  :  [ 13.9         53.87852523 -15.23      ]
        Efield [V/m] :  [-3678.30779216    26.07094068   854.47946414]
Force on dipole [N]  :  [ -3.03510764e-15  -1.14399221e-16  -9.61579111e-16]

    MS Postion [um]  :  [ 13.9         50.78756165 -15.23      ]
        Efield [V/m] :  [-3659.29045266    21.82336055   853.23524259]
Force on dipole [N]  :  [ -2.98781385e-15  -8.91551837e-17  -3.07613150e-17]

    MS Postion [um]  :  [ 13.9       47.617463 -15.23    ]
        Efield [V/m] :  [-3637.20564451     6.51425718   860.13378983]
Force on dipole [N]  :  [ -3.13051139e-15  -1.57805429e-16  -6.15215266e-17]

    MS Postion [um]  :  [ 13.9         44.37273415 -15.23      ]
        Efield [V/m] :  [-3645.14935177   -47.17909536   917.36247474]
Force on

    MS Postion [um]  :  [ 13.9        -79.49411454 -15.23      ]
        Efield [V/m] :  [-3453.82752197   -15.01416303   657.84847385]
Force on dipole [N]  :  [ -2.65733620e-15   1.82690878e-16  -1.20611766e-15]

    MS Postion [um]  :  [ 13.9        -81.95132428 -15.23      ]
        Efield [V/m] :  [-3401.9782076    -34.31383216   632.78986525]
Force on dipole [N]  :  [ -3.10414918e-15   6.73280551e-17  -9.04585500e-16]

    MS Postion [um]  :  [ 13.9        -84.29903986 -15.23      ]
        Efield [V/m] :  [-3399.04788037   -58.06180758   638.46776036]
Force on dipole [N]  :  [ -3.23265747e-15  -7.16440950e-17  -9.94653015e-16]

    MS Postion [um]  :  [ 13.9        -86.53392507 -15.23      ]
        Efield [V/m] :  [-3453.73228199   -56.46048794   670.94689303]
Force on dipole [N]  :  [ -3.15228084e-15  -2.38326870e-16  -1.33417723e-15]

    MS Postion [um]  :  [ 13.9      -88.652804 -15.23    ]
        Efield [V/m] :  [-3439.72389444   -18.00542233   704.49813684]
Force on dipol

    MS Postion [um]  :  [ 13.9        -76.05356772 -15.23      ]
        Efield [V/m] :  [-3516.23200559  -118.56255489   673.5812904 ]
Force on dipole [N]  :  [ -3.04893666e-15   5.07851900e-16   6.54601210e-17]

    MS Postion [um]  :  [ 13.9        -73.35471059 -15.23      ]
        Efield [V/m] :  [-3577.93944415   -17.52074085   727.67184763]
Force on dipole [N]  :  [ -2.87898541e-15   1.91604360e-17  -5.43703862e-16]

    MS Postion [um]  :  [ 13.9        -70.55857558 -15.23      ]
        Efield [V/m] :  [-3591.02008938   -24.68023752   690.58626274]
Force on dipole [N]  :  [ -2.76750322e-15  -2.37365298e-16  -7.88986495e-16]

    MS Postion [um]  :  [ 13.9        -67.66913613 -15.23      ]
        Efield [V/m] :  [-3641.26085189    31.88477968   668.73012357]
Force on dipole [N]  :  [ -4.50415533e-15  -2.61357966e-16  -5.50974764e-16]

    MS Postion [um]  :  [ 13.9        -64.69049831 -15.23      ]
        Efield [V/m] :  [-3524.04375984    23.98628862   672.56609738]
Force on

In [53]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(500), force_z3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(3,13)

shaking378_template = np.array(test_shaking378[1])
# shaking378_template = force_z3

fft_angles = np.angle(np.fft.rfft(shaking378_template))
fft_fft = np.abs(np.fft.rfft(shaking378_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
# fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
# tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases.append(phase_all)

gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

In [1]:
mm_gravity2_2 = get_alpha(0)

NameError: name 'get_alpha' is not defined

In [114]:
mm_gravity.fval, mm_gravity2.fval, mm_gravity2_2.fval

(129.02309291499253, 109.20476540830566, 109.22296265489145)

### Test new fitter

In [341]:
ll = likelihood_analyser.LikelihoodAnalyser()
def get_edm(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'error_A': 0.1, 'limit_A': [-10, 10],
                  'A2': 0.0, 'error_A2': 0.1, 'limit_A2': [-10, 10],
                  'A3': 0.0, 'error_A3': 0.1, 'limit_A3': [-10, 10],
                  'A4': 0.0, 'error_A4': 0.1, 'limit_A4': [-100, 100],
                  'sigma':13.85, 'error_sigma': 10, 'limit_sigma': [0, 1000000],
               'errordef': 1, 'print_level': 0, 
                  'fix_A': False, 'fix_A2': False, 'fix_A3': False, 'fix_A4': False}
    
#     fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
#               'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
#               'limit_phi': [0, np.pi],
#               'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
#               'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    noises_std = func_sideband(data_z2, gravity_freqs_all)
#     print(noises_std)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_edm_multiHarmoincs2(x=data_z2, amps1=edm_amp4, phases1=edm_phases4, 
                                         amps2=edm_amp5, phases2=edm_phases5, 
                                         amps3=edm_amp6, phases3=edm_phases6, 
                                         amps4=gravity_amp, phases4=gravity_phases, scales=gravity_scales,
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
        
    return mm_tmp.values[0], mm_tmp.values[1], mm_tmp.values[2], mm_tmp.values[3], mm_tmp.fval

In [333]:
len(gravity_amp), len(edm_amp), len(edm_amp2), len(edm_amp3)

(11, 11, 11, 11)

In [356]:
index = np.random.choice(11, 11, replace=False)
edm_amp4 = [edm_amp[i] for i in index]
index = np.random.choice(11, 11, replace=False)
edm_phases4 = [np.random.uniform()*2*np.pi for i in index]

In [253]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,13)[1:]

# edm_template = np.array(test_shaking378[1])
edm_template = force_z3

fft_angles = np.angle(np.fft.rfft(edm_template))
fft_fft = np.abs(np.fft.rfft(edm_template)) * 2 / np.sqrt(5000 * 500)*np.sqrt(10)
freq1 = np.fft.rfftfreq(len(edm_template), d=1./500)
# fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
# tf_z = BeadDataFile.BeadDataFile(fname)
# fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
# freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
# fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
# freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

edm_phases3 = []
edm_amp3 = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    edm_phases3.append(phase_all)
    edm_amp3.append(fft_fft[freq1==drive_freq])
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

In [79]:
mm_edm1 = get_edm(0)

MLEs:  <ValueView of Minuit at 31966e88>
  A: -0.18119256747340273
  A2: 0.0
  A3: 0.0
  sigma: 810.5023906881947


In [80]:
mm_edm1.fval, mm_edm2.fval, mm_edm3.fval

(172.7550549240229, 172.48574388265993, 172.4265137312733)

In [85]:
mm_edm1.values,mm_edm2.values,mm_edm3.values

(<ValueView of Minuit at 31966e88>
   A: -0.18119256747340273
   A2: 0.0
   A3: 0.0
   sigma: 810.5023906881947,
 <ValueView of Minuit at 2851b988>
   A: 0.37950323716012235
   A2: 4.538985305866177
   A3: 0.0
   sigma: 802.0055768287459,
 <ValueView of Minuit at 31196c68>
   A: 0.30447346179707324
   A2: 4.719080557733491
   A3: -0.27922596402177646
   sigma: 798.6539402335402)

In [357]:
mm_all_edm11 = np.array(Parallel(n_jobs=46)(delayed(get_edm)(i) for i in tqdm(range(10000))))




  0%|          | 0/10000 [00:00<?, ?it/s]


  0%|          | 46/10000 [00:02<10:19, 16.06it/s]


  1%|          | 92/10000 [00:08<13:00, 12.69it/s]


  1%|▏         | 138/10000 [00:16<18:17,  8.98it/s]


  2%|▏         | 184/10000 [00:23<19:17,  8.48it/s]


  2%|▏         | 230/10000 [00:31<22:08,  7.36it/s]


  3%|▎         | 276/10000 [00:38<23:35,  6.87it/s]


  3%|▎         | 322/10000 [00:46<24:43,  6.52it/s]


  4%|▎         | 368/10000 [00:54<25:28,  6.30it/s]


  4%|▍         | 414/10000 [01:01<25:21,  6.30it/s]


  5%|▍         | 460/10000 [01:09<25:17,  6.29it/s]


  5%|▌         | 506/10000 [01:16<25:25,  6.23it/s]


  6%|▌         | 552/10000 [01:24<25:33,  6.16it/s]


  6%|▌         | 598/10000 [01:32<25:39,  6.11it/s]


  6%|▋         | 644/10000 [01:39<25:24,  6.14it/s]


  7%|▋         | 690/10000 [01:47<25:57,  5.98it/s]


  7%|▋         | 736/10000 [01:53<24:07,  6.40it/s]


  8%|▊         | 782/10000 [02:00<23:59,  6.41it/s]


  8%|▊         | 828/10000 [02:09<24:

 59%|█████▉    | 5888/10000 [15:38<10:55,  6.27it/s]


 59%|█████▉    | 5934/10000 [15:46<10:45,  6.30it/s]


 60%|█████▉    | 5980/10000 [15:53<10:38,  6.30it/s]


 60%|██████    | 6026/10000 [16:01<10:44,  6.16it/s]


 61%|██████    | 6072/10000 [16:08<10:32,  6.21it/s]


 61%|██████    | 6118/10000 [16:15<10:13,  6.33it/s]


 62%|██████▏   | 6164/10000 [16:22<09:58,  6.41it/s]


 62%|██████▏   | 6210/10000 [16:29<09:45,  6.48it/s]


 63%|██████▎   | 6256/10000 [16:37<09:59,  6.24it/s]


 63%|██████▎   | 6302/10000 [16:44<09:55,  6.21it/s]


 63%|██████▎   | 6348/10000 [16:52<09:48,  6.21it/s]


 64%|██████▍   | 6394/10000 [17:00<10:10,  5.91it/s]


 64%|██████▍   | 6440/10000 [17:06<09:21,  6.34it/s]


 65%|██████▍   | 6486/10000 [17:14<09:10,  6.38it/s]


 65%|██████▌   | 6532/10000 [17:21<09:15,  6.24it/s]


 66%|██████▌   | 6578/10000 [17:28<08:56,  6.38it/s]


 66%|██████▌   | 6624/10000 [17:36<08:53,  6.32it/s]


 67%|██████▋   | 6670/10000 [17:43<08:50,  6.28it/s]


 67%|█████

In [117]:
edm1_all = [mm_.values[0] for mm_ in mm_all_only1]
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(len(edm1_all)), edm1_all)
ax.scatter(range(len(mm_all_only2)), mm_all_only2)
ax.scatter(range(len(mm_all_only3)), mm_all_only3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [524]:
edm1_all2 = [mm_.values[0] for mm_ in mm_all_only1 if (mm_.values[0]<2)&(mm_.values[0]>-2)]
edm2_all2 = [mm_ for mm_ in mm_all_only2 if (mm_<2)&(mm_>-2)]
edm3_all2 = [mm_ for mm_ in mm_all_only3 if (mm_<2)&(mm_>-2)]
_,ax = plt.subplots(1,2,figsize=(9.2,4))
_ = ax[0].hist(mm_all_edm[:,0], bins=50, range=(-2,2), label=r'$p_x$ scale')
_ = ax[0].hist(mm_all_edm[:,1], bins=20, alpha=0.7, range=(-2,2), label=r'$p_y$ scale')
_ = ax[0].hist(mm_all_edm[:,2], bins=50, alpha=0.4, range=(-2,2), label=r'$p_z$ scale')
ax[0].set(xlabel='simulation scale')
ax[0].legend()
_ = ax[1].hist(np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2), alpha=0.4, range=(0,5), bins=40, 
               label=r'$\sqrt{p_x^2+p_y^2+p_z^2}$')
ax[1].set(xlabel=r'absolute simulation scale')
ax[1].legend()

print(np.mean(mm_all_edm[:,0]), np.std(mm_all_edm[:,0]))
print(np.mean(mm_all_edm[:,1]), np.std(mm_all_edm[:,1]))
print(np.mean(mm_all_edm[:,2]), np.std(mm_all_edm[:,2]))
print(np.mean(np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2)))
print(np.std(np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0143668179208 0.555613566458
-0.0983347957548 1.24478125831
-0.0121948084302 0.297306993605
0.726948328118
1.19505198374


In [526]:
_,ax = plt.subplots(figsize=(9.2,4))
titles = [r'$p_x$ scale',r'$p_y$ scale',r'$p_z$ scale']
[ax.scatter(range(10000), mm_all_edm[:,i],label=titles[i]) for i in range(3)]
ax.legend()
ax.set(ylabel='simulation scale', xlabel='dataset')
# ax.scatter(range(10000), np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2))
# ax.scatter(range(1000), mm_all_edm3[:,3])
# ax.scatter(range(10000), mm_all_edm[:,4])
# ax.scatter(range(10000), mm_all_edm5[:,4])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.5, 'simulation scale'), Text(0.5, 0, 'dataset')]

In [184]:
mm_all_edm4.shape

(10000, 5)

In [359]:
alpha_edm = [alpha_ for alpha_,chi2 in zip(mm_all_edm[:,3],mm_all_edm[:,4]) if chi2<180]
alpha_edm5 = [alpha_ for alpha_,chi2 in zip(mm_all_edm5[:,3],mm_all_edm5[:,4]) if chi2<170]
alpha_edm11 = [alpha_ for alpha_,chi2 in zip(mm_all_edm11[:,3],mm_all_edm11[:,4]) if chi2<170]

In [360]:
len(alpha_edm11)

9705

In [527]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(len(alpha_edm)), alpha_edm, label='with edm')
ax.scatter(range(len(alpha_edm5)), alpha_edm5, alpha=0.7, label='w/o edm')
ax.scatter(range(len(alpha_edm11)), alpha_edm11, alpha=0.4, label='random edm')
ax.legend()
ax.set(ylabel='simulation scale', xlabel='dataset')
print('with edm (mean, ste): ',np.mean(mm_all_edm[:,3]), np.std(mm_all_edm[:,3])/np.sqrt(10000))
print('w/o edm (mean, ste): ',np.mean(mm_all_edm5[:,3]), np.std(mm_all_edm5[:,3])/np.sqrt(10000))
print('random edm (mean, ste): ',np.mean(mm_all_edm11[:,3]), np.std(mm_all_edm11[:,3])/np.sqrt(10000))
# print('with edm (mean, ste): ', np.mean(alpha_edm), np.std(alpha_edm)/np.sqrt(10000))
# print(np.mean(alpha_edm5), np.std(alpha_edm5)/np.sqrt(10000))
# print(np.mean(alpha_edm11), np.std(alpha_edm11)/np.sqrt(10000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

with edm (mean, ste):  0.314726936746 0.147261649134
w/o edm (mean, ste):  0.830508937732 0.112488776049
random edm (mean, ste):  0.864447292648 0.148614968952


In [529]:
_,ax = plt.subplots()
ax.hist(mm_all_edm5[:,3], range=(-25,25), bins=30, alpha=1, label='no edm')
ax.hist(mm_all_edm11[:,3], range=(-25,25), bins=30, alpha=0.7, label='random edm')
ax.hist(mm_all_edm[:,3], range=(-25,25), bins=30, alpha=0.3, label='with edm')
ax.legend()
ax.set(xlabel=r'$\alpha \, [10^{8}]$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, '$\\alpha \\, [10^{8}]$')]

### PL

In [495]:
index = np.random.choice(10000, 10000, replace=False)

In [619]:
ll = likelihood_analyser.LikelihoodAnalyser()

def calc_PL(i):

    ii = index[i]
    data_z2 = np.mean(test_phases[ii:(ii+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'error_A': 0.1, 'limit_A': [-10, 10],
                  'A2': 0.0, 'error_A2': 0.1, 'limit_A2': [-10, 10],
                  'A3': 0.0, 'error_A3': 0.1, 'limit_A3': [-10, 10],
                  'A4': 0.0, 'error_A4': 0.1, 'limit_A4': [-100, 100],
                  'sigma':13.85, 'error_sigma': 10, 'limit_sigma': [0, 1000000],
               'errordef': 1, 'print_level': 0, 
                  'fix_A': True, 'fix_A2': True, 'fix_A3': True, 'fix_A4': False}
    
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_edm_multiHarmoincs2(x=data_z2, amps1=edm_amp, phases1=edm_phases, 
                                         amps2=edm_amp2, phases2=edm_phases2, 
                                         amps3=edm_amp3, phases3=edm_phases3, 
                                         amps4=gravity_amp, phases4=gravity_phases, scales=gravity_scales,
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    

    fit_kwargs['A'] = mm_tmp.values[0]
    fit_kwargs['A2'] = mm_tmp.values[1]
    fit_kwargs['A3'] = mm_tmp.values[2]
    fit_kwargs['sigma'] = mm_tmp.values[4]
    fit_kwargs['fix_sigma'] = True
    
    PL_array = A_array*0
    PL_array = 2*10*ll.get_PL_edm_multiHarmonics(A_array=A_array, **fit_kwargs)
        
    return PL_array

In [620]:
A_array = np.arange(-1,1,0.05)
PLs_all_noEDM = []
for i in range(10):
    print('****************************')
    print(i)
    print('****************************')
    PLs_all_noEDM.append(np.array(Parallel(n_jobs=40)(delayed(calc_PL)(i) for i in tqdm(range(1000)))))









  0%|          | 0/1000 [00:00<?, ?it/s]

****************************
0
****************************










  4%|▍         | 40/1000 [00:03<01:35, 10.07it/s]







  8%|▊         | 80/1000 [00:07<01:29, 10.30it/s]







 12%|█▏        | 120/1000 [00:10<01:16, 11.52it/s]







 16%|█▌        | 160/1000 [00:13<01:09, 12.11it/s]







 20%|██        | 200/1000 [00:15<01:01, 12.98it/s]







 24%|██▍       | 240/1000 [00:18<00:55, 13.58it/s]







 28%|██▊       | 280/1000 [00:21<00:52, 13.72it/s]







 32%|███▏      | 320/1000 [00:23<00:47, 14.20it/s]







 36%|███▌      | 360/1000 [00:26<00:43, 14.55it/s]







 40%|████      | 400/1000 [00:28<00:40, 14.75it/s]







 44%|████▍     | 440/1000 [00:31<00:38, 14.49it/s]







 48%|████▊     | 480/1000 [00:34<00:36, 14.35it/s]







 52%|█████▏    | 520/1000 [00:37<00:33, 14.36it/s]







 56%|█████▌    | 560/1000 [00:40<00:30, 14.41it/s]







 60%|██████    | 600/1000 [00:42<00:27, 14.49it/s]







 64%|██████▍   | 640/1000 [00:45<00:23, 15.07it/s]







 68%|██████▊   | 680/1000 [00:48<00:22, 14.44it/s]







 72%|███

****************************
1
****************************










  8%|▊         | 80/1000 [00:02<00:30, 29.78it/s]







 12%|█▏        | 120/1000 [00:05<00:37, 23.68it/s]







 16%|█▌        | 160/1000 [00:07<00:41, 20.01it/s]







 20%|██        | 200/1000 [00:10<00:44, 18.11it/s]







 24%|██▍       | 240/1000 [00:13<00:44, 16.99it/s]







 28%|██▊       | 280/1000 [00:16<00:44, 16.09it/s]







 32%|███▏      | 320/1000 [00:18<00:42, 16.09it/s]







 36%|███▌      | 360/1000 [00:21<00:41, 15.49it/s]







 40%|████      | 400/1000 [00:23<00:38, 15.44it/s]







 44%|████▍     | 440/1000 [00:26<00:37, 14.92it/s]







 48%|████▊     | 480/1000 [00:29<00:35, 14.74it/s]







 52%|█████▏    | 520/1000 [00:32<00:31, 15.04it/s]







 56%|█████▌    | 560/1000 [00:35<00:30, 14.61it/s]







 60%|██████    | 600/1000 [00:37<00:26, 14.86it/s]







 64%|██████▍   | 640/1000 [00:40<00:24, 14.76it/s]







 68%|██████▊   | 680/1000 [00:43<00:21, 14.55it/s]







 72%|███████▏  | 720/1000 [00:46<00:19, 14.43it/s]







 76%|██

...









  8%|▊         | 80/1000 [00:02<00:33, 27.19it/s]







 12%|█▏        | 120/1000 [00:05<00:37, 23.25it/s]







 16%|█▌        | 160/1000 [00:07<00:42, 19.81it/s]







 20%|██        | 200/1000 [00:10<00:46, 17.27it/s]







 24%|██▍       | 240/1000 [00:13<00:44, 17.11it/s]







 28%|██▊       | 280/1000 [00:16<00:45, 15.89it/s]







 32%|███▏      | 320/1000 [00:18<00:43, 15.78it/s]







 36%|███▌      | 360/1000 [00:21<00:41, 15.58it/s]







 40%|████      | 400/1000 [00:24<00:39, 15.02it/s]







 44%|████▍     | 440/1000 [00:27<00:37, 14.75it/s]







 48%|████▊     | 480/1000 [00:30<00:35, 14.65it/s]







 52%|█████▏    | 520/1000 [00:32<00:32, 14.82it/s]







 56%|█████▌    | 560/1000 [00:35<00:29, 14.73it/s]







 60%|██████    | 600/1000 [00:38<00:27, 14.53it/s]







 64%|██████▍   | 640/1000 [00:40<00:24, 14.88it/s]







 68%|██████▊   | 680/1000 [00:43<00:22, 14.45it/s]







 72%|███████▏  | 720/1000 [00:46<00:19, 14.17it/s]







 76%|██

****************************
8
****************************










  8%|▊         | 80/1000 [00:02<00:33, 27.18it/s]







 12%|█▏        | 120/1000 [00:04<00:36, 24.34it/s]







 16%|█▌        | 160/1000 [00:08<00:44, 18.81it/s]







 20%|██        | 200/1000 [00:10<00:45, 17.55it/s]







 24%|██▍       | 240/1000 [00:13<00:44, 17.15it/s]







 28%|██▊       | 280/1000 [00:16<00:44, 16.21it/s]







 32%|███▏      | 320/1000 [00:18<00:43, 15.73it/s]







 36%|███▌      | 360/1000 [00:21<00:41, 15.59it/s]







 40%|████      | 400/1000 [00:24<00:40, 14.89it/s]







 44%|████▍     | 440/1000 [00:27<00:38, 14.73it/s]







 48%|████▊     | 480/1000 [00:29<00:35, 14.67it/s]







 52%|█████▏    | 520/1000 [00:32<00:31, 15.02it/s]







 56%|█████▌    | 560/1000 [00:35<00:29, 14.93it/s]







 60%|██████    | 600/1000 [00:37<00:26, 14.86it/s]







 64%|██████▍   | 640/1000 [00:40<00:24, 14.59it/s]







 68%|██████▊   | 680/1000 [00:43<00:22, 14.19it/s]







 72%|███████▏  | 720/1000 [00:46<00:19, 14.19it/s]







 76%|██

****************************
9
****************************










  8%|▊         | 80/1000 [00:02<00:32, 28.33it/s]







 12%|█▏        | 120/1000 [00:05<00:37, 23.62it/s]







 16%|█▌        | 160/1000 [00:07<00:41, 20.15it/s]







 20%|██        | 200/1000 [00:10<00:43, 18.45it/s]







 24%|██▍       | 240/1000 [00:13<00:44, 16.89it/s]







 28%|██▊       | 280/1000 [00:16<00:45, 15.66it/s]







 32%|███▏      | 320/1000 [00:18<00:43, 15.58it/s]







 36%|███▌      | 360/1000 [00:21<00:41, 15.39it/s]







 40%|████      | 400/1000 [00:24<00:39, 15.00it/s]







 44%|████▍     | 440/1000 [00:27<00:38, 14.59it/s]







 48%|████▊     | 480/1000 [00:30<00:35, 14.59it/s]







 52%|█████▏    | 520/1000 [00:32<00:32, 14.88it/s]







 56%|█████▌    | 560/1000 [00:35<00:30, 14.64it/s]







 60%|██████    | 600/1000 [00:38<00:27, 14.69it/s]







 64%|██████▍   | 640/1000 [00:40<00:24, 14.63it/s]







 68%|██████▊   | 680/1000 [00:43<00:22, 14.39it/s]







 72%|███████▏  | 720/1000 [00:46<00:19, 14.31it/s]







 76%|██

In [585]:
PLs_summed_harmonics = []
PLscopy = copy.deepcopy(PLs)
PLs_summed_harmonics = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics += pp_
PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_harmonics))

PLs_summed_harmonics2 = []
PLscopy = copy.deepcopy(PLs2)
PLs_summed_harmonics2 = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics2 += pp_
PLs_summed_harmonics2 -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_harmonics2))

In [586]:
0.43/np.sqrt(10)

0.13597793938724029

In [587]:
_,ax = plt.subplots()
# i=10
# [ax.plot(A_array, PLs[i], '*-') for i in range(100)]
ax.plot(A_array, PLs_summed_harmonics, '*-', label='with edm')
ax.plot(A_array, PLs_summed_harmonics2, '*-', label='no edm')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(-2,2), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood', title='files: 5000-6000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (-2.0, 2.0),
 Text(0.5, 0, '$\\alpha \\, [10^{8}]$'),
 Text(0, 0.5, 'Profile-Likelihood'),
 Text(0.5, 1.0, 'files: 5000-6000')]

In [588]:
type(PLs)

numpy.ndarray

In [621]:
PLlist = []
for i in range(10):
    PLlist += list(PLs_all_noEDM[i])

PLs_summed_noedm = []
PLscopy = copy.deepcopy(np.array(PLlist))
PLs_summed_noedm = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_noedm += pp_
PLs_summed_noedm -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_noedm))

PLlist2 = []
for i in range(10):
    PLlist2 += list(PLs_all_EDM[i])

PLs_summed_edm = []
PLscopy = copy.deepcopy(np.array(PLlist2))
PLs_summed_edm = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_edm += pp_
PLs_summed_edm -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_edm))


In [628]:
_,ax = plt.subplots()
ax.plot(A_array*10, PLs_summed_edm, '*-', label='with edm')
ax.plot(A_array*10, PLs_summed_noedm, '*-', label='no edm')
ax.legend()
ax.plot(A_array*10, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(-8,8), xlabel=r'$\alpha \, [10^{7}]$', ylabel = 'Profile-Likelihood', title='all files')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (-8.0, 8.0),
 Text(0.5, 0, '$\\alpha \\, [10^{7}]$'),
 Text(0, 0.5, 'Profile-Likelihood'),
 Text(0.5, 1.0, 'all files')]